In [1]:
import json

from agent.models.issue import Issue
from agent.models.mcphost import MCPHost
from agent.models.input import QueryFormatterInput

/Users/ikerochoa/opt/anaconda3/envs/kaizen-copilot-py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_issues(jsonl_path) -> list[Issue]:
    issues = []
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            try:
                sample = json.loads(line)
        
                issue_repo = sample.get("repo")
                repo_url = "https://github.com/" + issue_repo
                
                base_commit = sample.get("base_commit")
                issue_description = sample.get("problem_statement")
                if issue_description:
                    issues.append(Issue(title=f"{issue_repo} {base_commit}", description=issue_description, repo_url=repo_url))
            except json.JSONDecodeError:
                continue
    return issues

In [3]:
issues =  get_issues("./eval/swe_bench/data/swe_bench_lite_test.jsonl")

In [4]:
lenght_issues = len(issues)

In [5]:
lenght_issues

300

In [6]:
final_outputs = []

In [7]:
len(final_outputs)

0

In [8]:
final_list = issues[:]

for idx, issue in enumerate(final_list):
    print(f"Issue ({idx +1}/{len(final_list)}): {issue.title}")
    host = MCPHost()

    data = host.run_workflow("issue_resolution", initial_input=QueryFormatterInput(issue))
    final_outputs.append(data.fixed_code)
    print("Done!\n\n")

Issue (1/300): astropy/astropy d16bfe05a744909de4b27f5875fe0d4ed41ce607
Error al llamar a OpenAI: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-12-01-preview have exceeded token rate limit of your current AIServices S0 pricing tier. Please retry after 60 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}


RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-12-01-preview have exceeded token rate limit of your current AIServices S0 pricing tier. Please retry after 60 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}

In [ ]:
with open("predictions_agent_gpt_3.json", "w") as f:
    json.dump(final_outputs, f, indent=2)

In [ ]:
len(final_outputs)

300

In [ ]:
final_outputs[0]

'diff --git a/separable.py b/separable.py\nindex 3f37e909d..73fd88d5d 100644\n--- a/separable.py\n+++ b/separable.py\n@@ -107,11 +107,29 @@\n     if not model.separable:\n         # this does not work for more than 2 coordinates\n         mat = np.zeros((noutp, model.n_inputs))\n         if pos == "left":\n             mat[: model.n_outputs, : model.n_inputs] = 1\n         else:\n             mat[-model.n_outputs :, -model.n_inputs :] = 1\n     else:\n-        mat = np.zeros((noutp, model.n_inputs))\n-\n-        for i in range(model.n_inputs):\n-            mat[i, i] = 1\n-        if pos == "right":\n-            mat = np.roll(mat, (noutp - model.n_outputs))\n+        mat = np.zeros((noutp, model.n_inputs))\n+        for i in range(model.n_outputs):\n+            if model.n_inputs == model.n_outputs:\n+                mat[i, i] = 1\n+            elif model.n_inputs == 1:\n+                # Many outputs, one input: e.g., Linear1D(). Compound has n_outputs>1, n_inputs==1\n+             

## Read 

In [ ]:
data = []
with open("predictions_agent_gpt_3.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
len(data)

300

In [ ]:
def format_predictions_for_swebench(final_outputs):
    """
    Convierte una lista de predicciones al formato requerido por SWE-bench.

    :param final_outputs: lista de dicts con claves como 'instance_id' y 'patch' o similar
    :param model_name: nombre del modelo que se usará como valor de "model"
    :return: lista de predicciones formateadas
    """
    formatted = []
    
    with open("./eval/swe_bench/data/swe_bench_lite_test.jsonl", "r", encoding="utf-8") as f:
        for idx, line in enumerate(f):

            try:
                sample = json.loads(line)
        
                instance_id = sample.get("instance_id")
                formatted.append({
                    "instance_id": instance_id,
                    "model_name_or_path": "gpt-4.1",
                    "model_patch": final_outputs[idx]  # o 'model_patch' si ya viene así
                })
            except json.JSONDecodeError:
                continue
 
    return formatted

In [ ]:
final_list = format_predictions_for_swebench(data)

In [ ]:
len(final_list)

300

In [ ]:
with open("swe_bench_predictions_gpt_3.jsonl", "w", encoding="utf-8") as f:
    for item in final_list:
        json.dump(item, f)
        f.write("\n")

In [ ]:
# import json

# input_path = "swe_bench_predictions.jsonl"
# output_path = "swe_bench_predictions_cleaned.jsonl"

# with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
#     for line in infile:
#         line = line.strip()
#         if not line:
#             continue  # Saltar líneas vacías
#         try:
#             json_obj = json.loads(line)
#             json.dump(json_obj, outfile)
#             outfile.write("\n")
#         except json.JSONDecodeError:
#             print("❌ Línea inválida:", line[:100])  # O puedes hacer logging